# Tarea 02
Start up de bienes raices

- Objetivo: prototipa un modelo en Python que permita estimar el precio de una casa dadas algunas características que el usuario deberá proporcionar a través de un front al momento de la inferencia.

## Conjunto de Datos

Utilizaremos el conjuntos de datos de precios de compra-venta de casas de la ciudad Ames, Iowa en Estados Unidos disponible en [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques).